<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>

### 3.2 Model Building with Traffic
---


<div style="text-align: justify; line-height: 2">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Having observed that weather conditions do not have a significant impact on the end delay of STM buses, we will attempt to build a model that also includes traffic data to see if it can improve the model's predictive ability. The new model will include the same weather and bus data as the previous unfiltered model. This data will be combined with the traffic data from the City of Montreal, containing data points from November 26, 2019 to September 9, 2023. [See master + traffic dataset description](../Data/Dataset_descriptions.md). A sample of the data is shown below.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|     | line | delay_start | start_time | temp_start | rain_amt | snow_amt | delay_end | total_traffic |
| :-: | ---: | ----------: | ---------: | ---------: | -------: | -------: | --------: | ------------: |
|  0  |  467 |           0 |        405 |        7.1 |        0 |        0 |         6 |          3674 |
|  1  |  467 |           1 |        437 |        6.9 |        0 |        0 |         5 |          3116 |
|  2  |  467 |           1 |        468 |        6.9 |        0 |        0 |         6 |          3984 |
|  3  |  467 |          -1 |        496 |        6.5 |        0 |        0 |         6 |          2655 |
|  4  |  467 |           0 |        527 |        6.5 |        0 |        0 |         6 |          2273 |

|     | T_NBLT | T_NBT | T_NBRT | T_SBLT | T_SBT | T_SBRT | T_EBLT | T_EBT | T_EBRT | T_WBLT | T_WBT | T_WBRT |
| :-: | -----: | ----: | -----: | -----: | ----: | -----: | -----: | ----: | -----: | -----: | ----: | -----: |
|  0  |      0 |  1236 |     23 |      4 |  1270 |    155 |    165 |   157 |     68 |      2 |   478 |    116 |
|  1  |      0 |  1031 |     56 |      0 |   927 |    103 |    130 |   238 |     45 |      0 |   473 |    113 |
|  2  |      0 |  1330 |     44 |      0 |  1167 |    170 |    191 |   312 |     89 |      3 |   567 |    111 |
|  3  |      0 |   853 |     35 |      0 |   884 |    113 |    116 |   171 |     86 |      1 |   321 |     75 |
|  4  |      0 |   762 |     32 |      0 |   774 |     81 |     96 |   164 |     47 |      1 |   253 |     63 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.1: Sample of the data used for model building with traffic.</p>

</td>
</tr>
</table>

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|   Column Name   |                                    Description                                    |
| :-------------: | :-------------------------------------------------------------------------------: |
|     *line*      |                                  Bus line number                                  |
|  *delay_start*  |                    Delay at the start of the trip (in minutes)                    |
|  *start_time*   |         Time of day at the start of the trip (in minutes after midnight)          |
|  *temp_start*   |             Temperature at the start of the trip (in degrees Celsius)             |
|   *rain_amt*    |                   Rain amount at the start of the trip (in cm)                    |
|   *snow_amt*    |                   Snow amount at the start of the trip (in cm)                    |
|   *delay_end*   | Delay at the end of the trip (in minutes, negative values indicate early arrival) |
| *total_traffic* |               Total traffic during the trip (in number of vehicles)               |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.2: Description of the columns in the data set used for model building with traffic.</p>

</td>
</tr>
</table>

The coded columns represent the number of vehicles with a certain direction and turn at all the intersections during the trip. The first two letters represent the direction bound (N, S, E, W), and the last letters represent the turn (LT = left turn, T = through, RT = right turn). For example, T_NBLT represents the number of vehicles going through the intersection in the northbound direction and turning left. The total traffic is the sum of all the coded columns.

#### Unified Model

The correlation heatmap for the predictors is shown in Figure 3.2.1.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">
  
<p style="text-align: center;"><img src="../Images/3_2_1_corr_mat_3.png" alt="Figure 3.2.1" /></p>

<p style="text-align: center;"><strong><em>Figure 3.2.1: Correlation heatmap for the data set used for model building with traffic.</p>

</td>
</tr>
</table>

The correlation heatmap shows that the traffic data has some significant correlations with the delay at the end of the trip. The strongest correlations, excluding coded traffic columns, are with the total traffic (~0.17) and the delay at the start of the trip (~0.11). The correlations are still relatively weak, so we do not expect the traffic data to have definitive predictive ability. Note that the traffic at each direction and turn have greater correlations with the delay at the end of the trip than the total traffic. This is expected since bus routes have a fixed direction, so the traffic in the opposite direction should not affect the bus. However, the high correlations between all the coded traffic columns indicate that there is a lot of multicollinearity in these portions of the data. We will opt to use the total traffic as the sole traffic predictor to avoid multicollinearity issues. The model summary is shown in Table 3.2.2. 

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

| OLS Regression Results |                  |                         |           |
| :--------------------- | ---------------: | :---------------------- | --------: |
| **Dep. Variable:**     |        delay_end | **R-squared:**          |     0.050 |
| **Model:**             |              OLS | **Adj. R-squared:**     |     0.049 |
| **Method:**            |    Least Squares | **F-statistic:**        |     44.87 |
| **Date:**              | Mon, 25 Dec 2023 | **Prob (F-statistic):** |  8.69e-54 |
| **Time:**              |         14:42:26 | **Log-Likelihood:**     |   -17009. |
| **No. Observations:**  |             5134 | **AIC:**                | 3.403e+04 |
| **Df Residuals:**      |             5127 | **BIC:**                | 3.408e+04 |
| **Df Model:**          |                6 |                         |           |
| **Covariance Type:**   |        nonrobust |                         |           |

|                   |    coef |  std err |      t | P>&vert;t&vert; | [0.025 | 0.975] |
| :---------------- | ------: | -------: | -----: | --------------: | -----: | -----: |
| **delay_start**   |  0.0417 |    0.008 |  5.312 |               0 |  0.026 |  0.057 |
| **start_time**    |  0.0019 |        0 |  5.827 |               0 |  0.001 |  0.003 |
| **temp_start**    |  0.0433 |    0.012 |  3.549 |               0 |  0.019 |  0.067 |
| **rain_amt**      |  0.6417 |    0.129 |   4.96 |               0 |  0.388 |  0.895 |
| **snow_amt**      |  11.364 |   10.207 |  1.113 |           0.266 | -8.647 | 31.375 |
| **total_traffic** |  0.0004 | 2.75e-05 | 12.874 |               0 |      0 |      0 |
| **const**         | -1.6028 |    0.294 |  -5.45 |               0 | -2.179 | -1.026 |

|                    |          |                       |         |
| :----------------- | -------: | :-------------------- | ------: |
| **Omnibus:**       | 1201.053 | **Durbin-Watson:**    |   1.368 |
| **Prob(Omnibus):** |        0 | **Jarque-Bera (JB):** | 8628.43 |
| **Skew:**          |    0.927 | **Prob(JB):**         |       0 |
| **Kurtosis:**      |    9.075 | **Cond. No.**         |  474000 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.2: Model summary for the model built with all lines and traffic.</p>

</td>
</tr>
</table>

The model summary shows that snow amount is not significant with a p-value of 0.266, so it will be removed in the next model. The adjusted R-squared is 0.049, which is the same as the previous unfiltered model without traffic. However, the p-value for traffic is 0, which suggests that it is a significant predictor.

To further observe the relationship between the traffic and the response, we plot the scatter plots of the response against the predictors (including the coded columns) in Figure 3.2.2.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

<p style="text-align: center;"><img src="../Images/3_2_2_scatter_plots_3.png" alt="Figure 3.2.2" /></p>

<p style="font-weight: bold; font-style: italic; text-align: center;">Figure 3.2.2: Scatter plots of the response against the predictors for the data set used for model building with traffic.</p>

</td>
</tr>
</table>

As with the previous models, the scatter plots show that the relationship between the delay at the end of the trip and the delay at the start of the trip is non-linear, while the weather predictors have no apparent relationship with the response. The scatter plot of the total traffic shows a positive non-linear relationship between the delay at the end of the trip and the total traffic, which is more apparent by observing the scatter plots of the individual directions and turns. The shape of the distribution is close to exponential. 

#### Individual models

Since the total traffic might have a different effect on each bus line, we will build a model for each bus line. To ensure that the model is not overfitting, we will only build models for the bus lines with more than 300 data points. The number of data points for each bus line is shown in Table 3.2.5.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|    Line |    Count |
| ------: | -------: |
| **467** |  **395** |
|  **67** | **1218** |
|     480 |      171 |
| **121** | **1592** |
|      80 |      103 |
| **100** |  **789** |
| **460** |  **481** |
|     139 |       77 |
| **439** |  **308** |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.5: Number of data points for each bus line.</p>

</td>
</tr>
</table>

The bus lines with more than 300 data points are highlighted in bold. First, the models with no changes based on the previous observations in the unified model will be built. Then, the changes will be iteratively made to the model to observe the effect on the models' predictive ability. For simplicity, the whole model summary will not be shown for each bus line. The adjusted R-squared and the p-values for the predictors are shown in Table 3.2.3.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|                          | 467.0 |  67.0 | 121.0 | 100.0 | 460.0 | 439.0 |
| :----------------------- | ----: | ----: | ----: | ----: | ----: | ----: |
| **delay_start**          | 0.026 | 0.002 | 0.123 |     0 |     0 |     0 |
| **start_time**           | 0.261 |     0 |     0 |     0 |     0 |     0 |
| **temp_start**           | 0.005 | 0.014 | 0.091 |     0 |     0 | 0.359 |
| **rain_amt**             | 0.892 | 0.444 | 0.865 | 0.143 | 0.557 | 0.792 |
| **snow_amt**             | 0.555 | 0.232 | 0.786 |  0.01 | 0.002 | 0.688 |
| **total_traffic**        | 0.004 |     0 |     0 |     0 |     0 |     0 |
| **const**                | 0.382 | 0.255 | 0.001 |  0.01 | 0.001 | 0.098 |
|                          |       |       |       |       |       |       |
| **_Adjusted R-squared_** |  0.04 | 0.029 | 0.081 | 0.335 | 0.318 | 0.266 |



<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.3: Adjusted R-squared and p-values for the predictors for each bus line.</p>

</td>
</tr>
</table>

The adjusted R-squared values are all relatively low, with the highest being 0.335 for bus line 100. As shown in the unified model, snow amount is not significant for any of the bus lines, so it will be removed. The adjusted R-squared values for the models without snow amount are shown in Table 3.2.4.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|                          | 467.0 |  67.0 | 121.0 | 100.0 | 460.0 | 439.0 |
| :----------------------- | ----: | ----: | ----: | ----: | ----: | ----: |
| **delay_start**          | 0.026 | 0.002 | 0.123 |     0 |     0 |     0 |
| **start_time**           | 0.261 |     0 |     0 |     0 |     0 |     0 |
| **temp_start**           | 0.005 | 0.014 | 0.068 |     0 |     0 | 0.359 |
| **rain_amt**             | 0.892 | 0.444 | 0.867 | 0.143 | 0.557 | 0.792 |
| **total_traffic**        | 0.004 |     0 |     0 |     0 |     0 |     0 |
| **const**                | 0.382 | 0.255 | 0.001 |  0.01 | 0.001 | 0.098 |
|                          |       |       |       |       |       |       |
| **_Adjusted R-squared_** |  0.04 | 0.029 | 0.082 | 0.335 | 0.318 | 0.266 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.4: Adjusted R-squared and p-values for the predictors for each bus line without snow amount.</p>

</td>
</tr>
</table>

The adjusted R-squared did not decrease for any of the bus lines, so snow amount was removed fairly. The next predictor to be removed is rain amount. The adjusted R-squared values for the models without snow amount and rain amount are shown in Table 3.2.5.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|                          | 467.0 |  67.0 | 121.0 | 100.0 | 460.0 | 439.0 |
| :----------------------- | ----: | ----: | ----: | ----: | ----: | ----: |
| **delay_start**          | 0.025 | 0.002 | 0.123 |     0 |     0 |     0 |
| **start_time**           | 0.261 |     0 |     0 |     0 |     0 |     0 |
| **temp_start**           | 0.005 | 0.008 | 0.069 |     0 |     0 |  0.31 |
| **total_traffic**        | 0.003 |     0 |     0 |     0 |     0 |     0 |
| **const**                | 0.386 | 0.202 | 0.001 | 0.009 | 0.001 | 0.099 |
|                          |       |       |       |       |       |       |
| **_Adjusted R-squared_** | 0.042 | 0.029 | 0.082 | 0.334 | 0.319 | 0.268 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.5: Adjusted R-squared and p-values for the predictors for each bus line without snow amount and rain amount.</p>

</td>
</tr>
</table>

The adjusted R-squared did not decrease significantly for any of the bus lines, so rain amount was removed fairly. The remaining predictors cannot be removed for all the bus lines since their p-values are significant for some of the bus lines, so they will be kept. Since total_traffic appears to have an exponential relationship with the response, transformations will be applied to the predictor. The adjusted R-squared values for the models with the log transformation of total_traffic are shown in Table 3.2.6.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|                          | 467.0 |  67.0 | 121.0 | 100.0 | 460.0 | 439.0 |
| :----------------------- | ----: | ----: | ----: | ----: | ----: | ----: |
| **delay_start**          | 0.027 | 0.005 | 0.071 |     0 | 0.002 |     0 |
| **start_time**           | 0.287 |     0 |     0 |     0 |     0 |     0 |
| **temp_start**           | 0.009 |  0.03 | 0.439 |     0 |     0 | 0.844 |
| **total_traffic_log**    | 0.008 | 0.002 |     0 |     0 |     0 |     0 |
| **const**                | 0.076 | 0.072 |     0 |     0 |     0 | 0.365 |
|                          |       |       |       |       |       |       |
| **_Adjusted R-squared_** | 0.038 | 0.022 | 0.093 | 0.289 | 0.355 | 0.249 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.6: Adjusted R-squared and p-values for the predictors for each bus line with the log transformation of total_traffic.</p>

</td>
</tr>
</table>

The adjusted R-squared values increased for bus lines 121 and 460, but decreased for the other bus lines. A square root transformation will be applied to total_traffic instead. The adjusted R-squared values for the models with the square root transformation of total_traffic are shown in Table 3.2.7.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|                          | 467.0 |  67.0 | 121.0 | 100.0 | 460.0 | 439.0 |
| :----------------------- | ----: | ----: | ----: | ----: | ----: | ----: |
| **delay_start**          | 0.026 | 0.003 |  0.08 |     0 |     0 |     0 |
| **start_time**           | 0.276 |     0 |     0 |     0 |     0 |     0 |
| **temp_start**           | 0.006 | 0.015 | 0.157 |     0 |     0 | 0.469 |
| **total_traffic_sqrt**   | 0.006 |     0 |     0 |     0 |     0 |     0 |
| **const**                | 0.963 |  0.81 |     0 | 0.022 | 0.077 | 0.288 |
|                          |       |       |       |       |       |       |
| **_Adjusted R-squared_** |  0.04 | 0.025 | 0.094 |  0.32 |  0.34 |  0.26 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.7: Adjusted R-squared and p-values for the predictors for each bus line with the square root transformation of total_traffic.</p>

</td>
</tr>
</table>

The adjusted R-squared values increased for all bus lines except for bus line 100 when compared to the models with the log transformation of total_traffic. However, when compared to the models without any transformations, the adjusted R-squared values increased for bus lines 121 and 460, but decreased for the other bus lines. The square root transformation will be kept for all the bus lines. The final models for each bus line are shown in Table 3.2.8.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|                          |  467.0 |   67.0 |  121.0 |  100.0 |  460.0 |  439.0 |
| :----------------------- | -----: | -----: | -----: | -----: | -----: | -----: |
| **delay_start**          |  0.111 |  0.121 |  0.019 |   0.66 |  0.089 |  0.611 |
| **start_time**           | -0.001 | -0.002 |  0.003 |  0.005 |  0.016 | -0.005 |
| **temp_start**           |  0.108 |  0.065 | -0.021 | -0.726 | -1.419 |  0.123 |
| **total_traffic_sqrt**   |  0.038 |  0.031 |  0.056 |   0.38 |  0.483 |  0.087 |
| **const**                |  0.028 |  0.059 | -3.143 | -2.919 | -3.921 |  2.892 |
|                          |        |        |        |        |        |        |
| **_Adjusted R-squared_** |   0.04 |  0.025 |  0.094 |   0.32 |   0.34 |   0.26 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.8: Coefficients of final models for each bus line.</p>

</td>
</tr>
</table>

The adjusted R-squared values for the final models are all relatively low, with the highest being 0.34 for bus line 460. Thus, adding traffic data to the model did not yield a model capable of explaining a significant amount of the variance in the response. The models with significantly low adjusted R-squared values are bus lines 467, 67, and 121 ($R^2_{adj}<-0.100$). Considering that the p-values for their coefficients are also not significant, conclusions cannot be drawn from these models. The models with adjusted R-squared values greater than 0.100 are bus lines 100, 460, and 439. The p-values for the coefficients of these models are significant, with an exception of the temperature at the start of the trip for bus line 439. A prediction is made using these three models below.

The prediction is made for the following values:

(add predictions) 
</div>
&nbsp;  
&nbsp;  

---

<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<span style="flex: 1; text-align: center;">3.2 Model Building with Traffic</span>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>
